# Warning: Never test for equality of floats!

In [1]:
0.1 + 0.2 == 0.3

False

???? It's because of rounding errors in float operations

In [2]:
print 0.1 + 0.2

0.3


Can't trust anybody these days!

Similarly:

\begin{equation} 
\cos(60^o)=\sin(30^o)
\end{equation} 

but:

In [3]:
import math
print math.cos(math.pi/3) == math.sin(math.pi/6)
print math.cos(math.pi/3)
print math.sin(math.pi/6)

False
0.5
0.5


### What you should do instead

Check to see if difference is very small:


In [24]:
def are_equal(x, y, epsilon=1e-10): 
    return abs(x - y) < epsilon
print are_equal(0.3, 0.1 + 0.2)
print are_equal(math.cos(math.pi/3), math.sin(math.pi/6))

True
True


In [25]:
are_equal(0.3, 0.1 + 0.2, epsilon=1e-20)

False


# Mutable vs Immutable types in Python

This is something technical about how Python works but the technicality is very important to implement Gaussian elimination. 


In [6]:
# guess what will happen
x = 1
y = x
print 'x = ', x, '; y = ', y
x += 1
print 'x = ', x, '; y = ', y

x =  1 ; y =  1
x =  2 ; y =  1


Even though we said `y=x`, and then changed `x`, the value of `y` didn't change.

Let's try the same with lists:

In [7]:
x = [1,2,3,4]
y = x
print 'x = ', x, '; y = ', y
x[0] = 999
print 'x = ', x, '; y = ', y

x =  [1, 2, 3, 4] ; y =  [1, 2, 3, 4]
x =  [999, 2, 3, 4] ; y =  [999, 2, 3, 4]


Woah! What is going on? We saw one behaviour for numbers, and a totally different behaviour for lists.

This is because numbers are immutable whereas lists are mutable.

### Mechanics of mutable vs immutable (optional)

More concretely, when we say `x=1`, in the memory of the computer, a location (object) is created that stores the number `1`. A separate location, which corresponds to the label/identifier `x` is also created, but that location does not contain `1`. Instead, the location for `x` contains the address of the location in memory where `1` is stored. We can think of `x` as a pointer to the actual place in memory where `1` is stored. `y` also points to `1`.


In [8]:
# objects pointed to by x and y are identical:
x = 1
y = x
print id(x)
print id(y)

140589316225096
140589316225096


However, when some arithmetic is done on `x`, the result is stored in a new place in memory, to which `x`, but not `y`, points. In other words, you cannot mutate the object `1`.

In [9]:
# cannot change 1 'in-place'
x += 1 
print id(x)
print id(y)

140589316225072
140589316225096


On the other hand, when we do something to a list, it is modified in-place (mutated), so that any identifiers (such as `x` and `y`) that were pointing to it, continue to do so.

For example, writing `x = [1,2,3,4]` and then `y=x`, makes `x` and `y` point to the same `[1,2,3,4]` in memory. 



In [10]:
x = [1,2,3,4]
y = x
print id(x)
print id(y)

4594293864
4594293864


This is the same as for the `int` example. Lists and ints deviate in their behavior when we change the list, e.g. `x[0]=999`, a new list is not created (whereas a new int would have been):



In [11]:
x[0] = 999
print id(x)
print id(y)

4594293864
4594293864


Instead, the old list is mutated and this can be seen by examining `x` or `y`:


In [12]:
print x
print y

[999, 2, 3, 4]
[999, 2, 3, 4]


### Immutable vs Mutable objects as function arguments

In [13]:
# what will happen?
def f(a):
    a += a
    return a

x = 1
print f(x)
print x

2
1




The value of `x` didn't change because `a` was a local variable inside the function.

Let's try the same with lists:


In [14]:
# what will happen?
def f(a):
    a += a
    return a

x = [1,2,3,4]
print f(x)
print x

[1, 2, 3, 4, 1, 2, 3, 4]
[1, 2, 3, 4, 1, 2, 3, 4]




The value of `x` did change!!! This was because `x` was a list and lists are mutable. Which meant that when `x` was copied to `a` in the function, `a` now pointed to the exact same memory location as `x`, and therefore changing `a` amounts to changing `x`.

This behavior can actually be a good thing: it means that we can manipulate a list inside a function. 

So what if `x = [1,2,3,4`] and we really want a copy of `[1,2,3,4]` that will be different? Easy:

In [15]:
import copy
x = [1,2,3,4]
y = copy.copy(x)
print x, y
x[0] = 999
print x, y

[1, 2, 3, 4] [1, 2, 3, 4]
[999, 2, 3, 4] [1, 2, 3, 4]


**Mutable types:**

* Lists
* Tuples 
* Dictionaries 
* Numpy arrays

**Immutable types:**

* Integers, floats,
* bools
* Strings


# Swapping rows

Consider the following code, which is supposed to swap two rows of a matrix: 

In [27]:
import numpy as np 

Aug = np.array([[1,2],[3,4]])
print 'before row swap:'
print Aug 

dum = Aug[1,:]
Aug[1,:] = Aug[0,:]
Aug[0,:] = dum 

print 'after row swap:'
print Aug

before row swap:
[[1 2]
 [3 4]]
after row swap:
[[1 2]
 [1 2]]


Not what you expected! What went wrong? Use `print` to find out! 

In [28]:
Aug = np.array([[1,2],[3,4]])

dum = Aug[1,:]
print 'dum before changing Aug:'
print dum
Aug[1,:] = Aug[0,:]
print 'dum after changing Aug:'
print dum 

dum before changing Aug:
[3 4]
dum after changing Aug:
[1 2]


This is exactly what we saw earlier: since `dum` points to the same vector as `Aug[1,:]`, later changes to `Aug[1,:]` also change `dum`! What we need to do is to make `dum` an independent copy of `Aug[1,:]`. We can do this via `copy`: 

In [29]:
Aug = np.array([[1,2],[3,4]])
print 'Aug before row swap:'
print Aug 

dum = Aug[1,:].copy() # notice copy here!
Aug[1,:] = Aug[0,:]
Aug[0,:] = dum 

print 'Aug after row swap'
print Aug

Aug before row swap:
[[1 2]
 [3 4]]
Aug after row swap
[[3 4]
 [1 2]]


Great! We are now ready to implement Gaussian elimination...

# Exercise 1: Gaussian Elimination with partial pivoting

Modify your Gaussian elimination code for HW04, if necessary, so that it
* carries out partial pivoting 
* works on an arbitrary $n \times m$ matrix [not just $n \times (n+1)$] 

[Hint: see Algo 6.2 of Text]

In [23]:
import numpy as np 

def number_rows(Aug):     
    n, m = Aug.shape     
    return n

def swap(Aug, i, p):  
    dum = Aug[p,:].copy()
    Aug[p,:] = Aug[i,:]
    Aug[i,:] = dum

def eliminate_lower_column(Aug, i):
    # your code here 
    pass 

def is_zero(x, epsilon=1e-10):    
    return abs(x) < epsilon
    
def GEpp(Aug, TOL=1e-6):    
    Aug = np.array(Aug, dtype=np.float32)
    n = number_rows(Aug)
    for i in range(n-1):
        p = np.argmax(abs(Aug[i:,i])) + i
        # your code here 
    pass 

## Exercise 2: Replacing $b$ with $I$

Apply the code you wrote in Exercise 1 to an augmented matrix resulting from the concatenation of a matrix $A$ (specified below) and the identity matrix $I$: 

In [24]:
A = np.array([[1,1,-1],
             [1,1,0],
             [-1,1,2]])
print 'A = '
print A 
print

I = np.identity(3)
print 'I = '
print I 
print

AUG = np.concatenate((A, I), axis=1)
print 'Augmented matrix [A, I] = '
print AUG 
print

print 'Gaussian Elimination with partial pivoting applied to [A, I] is:'
print GEpp(AUG)

A = 
[[ 1  1 -1]
 [ 1  1  0]
 [-1  1  2]]

I = 
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]

Augmented matrix [A, I] = 
[[ 1.  1. -1.  1.  0.  0.]
 [ 1.  1.  0.  0.  1.  0.]
 [-1.  1.  2.  0.  0.  1.]]

Gaussian Elimination with partial pivoting applied to [A, I] is:
None


Your code should return this matrix:

```
[[ 1.  1. -1.  1.  0.  0.]
 [ 0.  2.  1.  1.  0.  1.]
 [ 0.  0.  1. -1.  1.  0.]]
```


# Exercise 3: Eliminate last column

Having zero'd the elements of $A$ that lie below the diagonal, we will now do so for those elements above the diagonal. We will begin, though, by just doing this for the last column of $A$. 

Write a new function `GJ_eliminate_last_column(Aug)` that calls `GEpp(Aug)` and then zeros the last column of $A$ using elementary row operations. It should return a matrix of the form $[U,M]$ where the last column of $U$ has only a one in its last entry, i.e. the last column of $U$ is the same as the last column of the identity matrix. 

Test your function on the augmented matrix constructed above. 

In [26]:
def scale(Aug, i): 
    Aug[i,:] /= Aug[i,i]
    
def eliminate_upper_column(Aug, i): 
    # your code here 
    pass 

def GJ_eliminate_last_column(Aug, TOL=1e-6):
    # your code here 
    pass 

print 'GJ_eliminate_last_column:'
print GJ_eliminate_last_column(AUG)

GJ_eliminate_last_column:
None


Your code should return: 
```
[[ 1.  1.  0.  0.  1.  0.]
 [ 0.  2.  0.  2. -1.  1.]
 [ 0.  0.  1. -1.  1.  0.]]
 ```

# Exercise 4: Eliminate all columns

Copy your `GJ_eliminate_last_column(Aug)` function to a new function `GJ(Aug)`.
Now modify this function so that it returns a matrix of the form $[I,M]$ where $I$ is the identity matrix using only elementary row operations. Apply this to the same matrix as above.


In [27]:
def GJ(Aug, TOL=1e-6):
    # your code here 
    pass 
print 'GJ:'
print GJ(AUG)

GJ:
None


Your code should return 
```
[[ 1.   0.   0.  -1.   1.5 -0.5]
 [ 0.   1.   0.   1.  -0.5  0.5]
 [ 0.   0.   1.  -1.   1.   0. ]]
 ```

# Exercise 5: Fetch $M$

Your function returns a matrix of the form $[I,M]$. Fetch $M$ and compute $AM$ and $MA$. What do you find?


In [30]:
# your code here
print 'A*M = '
# your code here
print 'M*A = '
# your code here

A*M = 
M*A = 


Congratulations! You have just implemented the Gauss-Jordan method for computing the inverse of a matrix. Well done! 